In [201]:
import numpy as np

In [253]:
GRID_SIZE = 3 ** 2
NB_INDIVIDUALS_POPULATION = 4
FITNESS_GOAL = GRID_SIZE * GRID_SIZE * 2

In [203]:
def generate_individual():
    """
    Initialize a random Sudoku grid where each cell has an equal chance of being a number from 1 to 9.
    So the grid can be invalid. But aftern, we will use a genetic to make the grid valid.

    Returns:
        A numpy.ndarray with dimensions (GRID_SIZE, GRID_SIZE) where each cell contains a value between 1 and GRID_SIZE (inclusive).
    """
    min_value = 1
    max_value = GRID_SIZE + 1
    size = (GRID_SIZE, GRID_SIZE)
    return np.random.randint(min_value, max_value, size=size)


In [255]:
generate_individual()

array([[9, 1, 6, 5, 1, 7, 7, 4, 4],
       [9, 9, 3, 1, 6, 4, 4, 7, 3],
       [9, 4, 7, 5, 8, 8, 1, 6, 1],
       [7, 6, 4, 3, 7, 3, 3, 5, 8],
       [2, 3, 1, 7, 4, 2, 9, 9, 5],
       [3, 6, 7, 6, 4, 8, 8, 3, 8],
       [6, 9, 9, 9, 3, 7, 4, 7, 8],
       [2, 3, 8, 6, 8, 2, 3, 7, 1],
       [1, 3, 2, 5, 4, 7, 6, 2, 6]])

In [205]:
def generate_population():
    """
    Generates a population by creating a list of individuals using the generate_individual function.

    Returns:
        list: A population consisting of multiple individuals.
    """
    return [generate_individual() for x in range(NB_INDIVIDUALS_POPULATION)]

In [206]:
def compute_fitness(individual):
    """
    Computes the fitness score of an individual.

    Args:
        individual (numpy.ndarray): A numpy.ndarray with dimensions (GRID_SIZE, GRID_SIZE) where each cell contains a value between 1 and GRID_SIZE (inclusive).

    Returns:
        int: Fitness score of the individual.
    """
    fitness = 0

    for i in range(GRID_SIZE):
        unique_digit_column = np.unique(individual[:, i])
        unique_digit_line = np.unique(individual[i])
        fitness += len(unique_digit_line)
        fitness += len(unique_digit_column)
        
    return fitness

In [207]:
def selection_by_wheel(population, fitness_scores):
    """
    Selects 50% of the population based on their fitness scores using roulette wheel selection.

    Roulette wheel selection is a widely used method in genetic algorithms for selecting individuals from a population based on their fitness scores. The selection process is analogous to a roulette wheel, where each individual's probability of selection is proportional to their fitness score. Higher fitness scores correspond to larger areas on the roulette wheel, increasing the likelihood of selection.

    Args:
        population (list): List of grid in the current population.
        fitness_scores (list): List of fitness scores corresponding to the individuals/population.

    Returns:
        list: Selected grids based on the selection probabilities.
    """
    nb_individual_to_select = len(population) // 2  

    total_fitness = sum(fitness_scores)
    selection_probabilities = [fit / total_fitness for fit in fitness_scores]

    selected_indices = np.random.choice(np.arange(len(population)), size=nb_individual_to_select, p=selection_probabilities)
    selected_individuals = [population[i] for i in selected_indices]

    return selected_individuals

In [250]:
def selection_by_rank(population, fitness_scores):
    """
    Selects 50% of the population based on their fitness scores using rank-based selection.

    Rank-based selection selects individuals based on their position in the fitness ranking.
    Individuals with higher fitness scores have a higher chance of being selected.

    Args:
        population (list): List of grid in the current population.
        fitness_scores (list): List of fitness scores corresponding to the individuals/population.

    Returns:
        list: Selected grids based on the selection probabilities.
    """
    nb_individual_to_select = len(population) // 2  

    ranked_indices = np.argsort(fitness_scores)
    selected_indices = ranked_indices[-nb_individual_to_select:][::-1]
    
    selected_individuals = [population[i] for i in selected_indices]

    return selected_individuals


In [209]:
def crossover(grid1, grid2):
    """
    Performs crossover between two Sudoku grids to create two descendants.

    Args:
        parent1 (numpy.ndarray): First grid.
        parent2 (numpy.ndarray): Second grid.

    Returns:
        tuple: Two descendants resulting from the crossover.
    """
    descendant1 = np.copy(grid1)
    descendant2 = np.copy(grid2)

    for i in range(GRID_SIZE):
        if i % 2 != 0:
            descendant1[i] = grid2[i]
            descendant2[i] = grid1[i]

    return descendant1, descendant2

In [210]:
def calculate_mutation_rate(fitness_score):
    """
    Calculates the mutation rate based on the fitness score.

    Args:
        fitness_score (int): Fitness score of a grid (higher score indicates a better solution).

    Returns:
        float: Calculated mutation rate.
    """
    return 1.0 - fitness_score / FITNESS_GOAL

In [211]:
def mutate(grid):
    """
    Mutates a Sudoku grid by randomly modifying some cells.

    Args:
        grid (numpy.ndarray): Sudoku grid to be mutated.

    Returns:
        numpy.ndarray: Mutated Sudoku grid.
    """
    mutated_grid = np.copy(grid)

    fitness_score = compute_fitness(mutated_grid)
    mutation_rate = calculate_mutation_rate(fitness_score)

    for i in range(GRID_SIZE):
        for j in range(GRID_SIZE):
            if np.random.random() < mutation_rate:
                other_possible_values = [num for num in range(1, GRID_SIZE + 1) if num not in grid[i]]
                if len(other_possible_values) != 0:
                    new_value = np.random.choice(other_possible_values)
                else:
                    new_value =np.random.choice(np.arange(1, GRID_SIZE + 1))
                mutated_grid[i, j] = new_value

    return mutated_grid

In [212]:
def generate_new_generation(current_population):
    """
    Generates a new generation of individuals based on the provided functions.

    Args:
        current_population (list of numpy.ndarray): Current population of individuals.

    Returns:
        list of numpy.ndarray: New generation of individuals.
    """
    fitness_scores = [compute_fitness(grid) for grid in current_population]

    parents = selection_by_rank(current_population, fitness_scores)

    num_children = len(current_population) - len(parents)

    children = []
    for i in range(num_children // 2):
        idx_parent1, idx_parent2 = np.random.choice(len(parents), size=2, replace=False)
        parent1 = parents[idx_parent1]
        parent2 = parents[idx_parent2]
        child1, child2 = crossover(parent1, parent2)
        children.append(child1)
        children.append(child2)

    mutated_children = [mutate(child) for child in children]

    new_generation = parents + mutated_children

    return new_generation

In [213]:
def check_solution(population):
    """
    Check if the solution has been found in the population.

    Args:
        population (list): List of grids in the current population.

    Returns:
        tuple: A tuple containing a flag indicating whether more generations are needed,
               and the best grid found in the population.
            - bool: True if more generations are needed, False otherwise.
            - numpy.ndarray: Best grid found from the population.
    """
    best_fitness = -1
    best_grid = None

    for grid in population:
        fitness = compute_fitness(grid)
        if fitness == FITNESS_GOAL:
            print("One possible sudoku grid is :", grid, sep="\n")
            return False, grid

        if fitness > best_fitness:
            best_fitness = fitness
            best_grid = grid

    print('MAX fitness score', best_fitness)
    return True, best_grid

In [251]:
def main():
    population = generate_population()

    need_more_generation = True
    i = 0
    while need_more_generation:
        print(f'Gen n°{i}')
        population = generate_new_generation(population)

        need_more_generation, best_grid = check_solution(population)
        i+=1

In [254]:
if __name__ == "__main__":
    main()

Gen n°0
MAX fitness score 120
Gen n°1
MAX fitness score 120
Gen n°2
MAX fitness score 120
Gen n°3
MAX fitness score 124
Gen n°4
MAX fitness score 124
Gen n°5
MAX fitness score 124
Gen n°6
MAX fitness score 124
Gen n°7
MAX fitness score 124
Gen n°8
MAX fitness score 125
Gen n°9
MAX fitness score 125
Gen n°10
MAX fitness score 125
Gen n°11
MAX fitness score 125
Gen n°12
MAX fitness score 125
Gen n°13
MAX fitness score 125
Gen n°14
MAX fitness score 125
Gen n°15
MAX fitness score 125
Gen n°16
MAX fitness score 125
Gen n°17
MAX fitness score 125
Gen n°18
MAX fitness score 125
Gen n°19
MAX fitness score 125
Gen n°20
MAX fitness score 125
Gen n°21
MAX fitness score 126
Gen n°22
MAX fitness score 126
Gen n°23
MAX fitness score 126
Gen n°24
MAX fitness score 126
Gen n°25
MAX fitness score 126
Gen n°26
MAX fitness score 126
Gen n°27
MAX fitness score 126
Gen n°28
MAX fitness score 126
Gen n°29
MAX fitness score 126
Gen n°30
MAX fitness score 126
Gen n°31
MAX fitness score 126
Gen n°32
MAX fitne